# 02. データをフォーマットする

【このノートで実施すること】</br>
　データがテキストファイルのままでは扱いづらいため、</br>
　JRDB指定のフォーマットに従って整形し、 Excelのような行と列で構成される形式へ変換します。</br>
</br>
【入力】</br>
　テキストファイル</br>
　・data/01_unzip_data/hjc 内の各テキストファイル</br>
　・data/01_unzip_data/kyi 内の各テキストファイル</br>
　・data/01_unzip_data/sed 内の各テキストファイル</br>
 
　フォーマットの情報</br>
　・data/02_format_data/format_info/hjc_info.json</br>
　・data/02_format_data/format_info/kyi_info.json</br>
　・data/02_format_data/format_info/sed_info.json</br>
</br>
　※フォーマットの情報はJRDBサイトの仕様書からあらかじめ作成してあリます。</br>
　　引用：</br>
　　　HJC：http://www.jrdb.com/program/Hjc/hjcdata_doc.txt</br>
　　　KYI：http://www.jrdb.com/program/Kyi/kyi_doc.txt</br>
　　　SED：http://www.jrdb.com/program/Sed/sed_doc.txt</br>
</br> 
　※もしhjc, kyi, sed 以外のデータを使用したい場合、同様の形式でjsonファイルを作成すればフォーマットできると思います。</br>
　※以降の処理で使わなそうな値についてはフォーマットの段階で除外するようにしています。もし全部の値をフォーマットしたい場合、</br>
  　　hjc_info_full.jsonなど、fullと書いてある方のファイルを指定してフォーマットしてください。</br>
　　ただし、これ以降の処理ではfullと書いてない方のファイルでフォーマットした前提で書いてありますので、適宜処理を追加する必要が生じます。</br>
</br>
【出力】</br>
　フォーマットされたファイル</br>
　・data/02_format_data/df_hjc.pkl</br>
　・data/02_format_data/df_kyi.pkl</br>
　・data/02_format_data/df_sed.pkl</br>
</br>

In [ ]:
# 必要なモジュールをインポート
import json
import os
import pandas as pd
import pickle
from tqdm import tqdm

In [ ]:
# データ整形用メソッド
def format_data(prefix, input_folder, output_folder, info_file, save_file_name):
    
    # フォルダ内のテキストファイル一覧を取得
    txt_files = get_text_files_in_subfolders(input_folder, prefix)

    # テキストファイルの数だけループする
    records = []
    for txt_file in tqdm(txt_files, desc="Processing files"):
        formatted_data = exec_format(txt_file, info_file)
        records.extend(formatted_data)
    
    # レコードリストをPandas DataFrameに変換
    df = pd.DataFrame(records)

    # 保存する
    save_dataframe_to_pickle(df, output_folder, save_file_name)

# フォルダ内のテキストファイルのパス一覧を取得する
# format_dataメソッド内で呼び出す
def get_text_files_in_subfolders(folder_path, prefix):
    txt_file_paths = []
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    
    for subfolder in subfolders:
        subfolder_path = os.path.join(folder_path, subfolder)
        txt_files = [f for f in os.listdir(subfolder_path) if f.startswith(prefix) and f.endswith('.txt')]
        
        for txt_file in txt_files:
            txt_file_paths.append(os.path.join(subfolder_path, txt_file))

    return txt_file_paths

# フォーマットの処理を実際に行うメソッド
# format_dataメソッド内で呼び出す
def exec_format(data_file, info_file):
    # JSONファイルからカラム情報を読み込む
    with open(info_file, 'r') as json_file:
        column_info = json.load(json_file)
    
    # テキストファイルを読み込む
    with open(data_file, 'r', encoding='cp932') as txt_file:
        data = txt_file.read()

    # テキストデータを改行文字で分割し、各行をレコードとして扱う
    lines = data.strip().split('\n')  # 最終行の改行文字を削除してから分割

    # 各レコードごとにデータを整形する
    records = []
    for line in lines:
        formatted_data = {}
        # 各カラムごとにデータを取得する
        for column_name, column_ranges in column_info["データ位置"].items():
            start_bit = column_ranges['start_bit']
            end_bit = column_ranges['end_bit']
            
            # ビット範囲に基づいてデータを取得
            column_value = line[start_bit:end_bit + 1]
            # print("column_value: ", column_value)
            
            # データを格納
            formatted_data[column_name] = column_value
        
        records.append(formatted_data)
    return records

# ファイルをpickle形式で保存する
# format_dataメソッド内で呼び出す
def save_dataframe_to_pickle(dataframe, folder_path, file_name):
    
    print("フォーマットしたデータをファイルに保存します")

    # フォルダが存在しない場合は作成
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # ファイルのフルパスを構築
    file_path = os.path.join(folder_path, file_name + ".pkl")

    # DataFrame を pickle 形式で保存
    with open(file_path, 'wb') as file:
        pickle.dump(dataframe, file)

    print("保存しました")

In [ ]:
# hjcファイルをフォーマットする
prefix = "HJC"
input_folder = "data/01_unzip_data/hjc"
output_folder = "data/02_format_data"
info_file = "data/02_format_data/format_info/hjc_info.json"
save_file_name = "df_hjc"
format_data(prefix, input_folder, output_folder, info_file, save_file_name)

In [ ]:
# kyiファイルをフォーマットする
prefix = "KYI"
input_folder = "data/01_unzip_data/kyi"
output_folder = "data/02_format_data"
info_file = "data/02_format_data/format_info/kyi_info.json"
save_file_name = "df_kyi"
format_data(prefix, input_folder, output_folder, info_file, save_file_name)

In [ ]:
# sedファイルをフォーマットする
prefix = "SED"
input_folder = "data/01_unzip_data/sed"
output_folder = "data/02_format_data"
info_file = "data/02_format_data/format_info/sed_info.json"
save_file_name = "df_sed"
format_data(prefix, input_folder, output_folder, info_file, save_file_name)